# Subscriber Cancellation Pipeline

This notebook demonstrates the process the data will undertake during the importing and cleaning of subscribe cancellation data. This process will then be created in pipeline.py, to be full automated.

In [21]:
import ast
import sqlite3
import numpy as np
import pandas as pd
import datetime as dt

## Connect to Database

In [22]:
# connect to database
con = sqlite3.connect("data/cademycode.db")
cursor = con.cursor()

# identify tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('cademycode_students',), ('cademycode_courses',), ('cademycode_student_jobs',)]


## Import Tables to Pandas Dataframes

In [23]:
# Extract data from students table
students_df = pd.read_sql_query("SELECT * FROM cademycode_students", con)
print('students:', students_df.shape)

# Extract data from courses table
courses_df = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
print('courses:', courses_df.shape)

# Extract data from student_jobs table
student_jobs_df = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)
print('student jobs:', student_jobs_df.shape)

students: (5000, 9)
courses: (10, 3)
student jobs: (13, 3)


## Students Table

In [24]:
# look at head
students_df.head()

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64


In [25]:
# Look at datatypes
students_df.dtypes

uuid                       int64
name                      object
dob                       object
sex                       object
contact_info              object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
dtype: object

In [26]:
# Look at info
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


**Observations:**
- `uuid` field acts as a unique id for each record, maintaining data integrity.
- `name` field includes both first name and last name.
    - May be better splitting into `first_name` and `last_name` fields.
- `dob` field appears to be the date of birth for the record.
    - might be better to change name to `date_of_birth` as it is more descriptive.
- `contact_info` appears to be a dictionary.
    - Each key, value pair should be its own field.
- `job_id`, `num_course_taken`, `current_career_path_id` appear to all be floats, but should be integers.
- `time_spent_hrs` is a float, but 
    - might be better off being put into a time datatype for better readability.

### Calculating Age

Calculating age would allow analysts to easily be able to analyse certain aspects of the unsubscribers based on their age. It would create an ordinal field that the analysts could explore.

In [27]:
# Convert date of birth to datetime
students_df['dob'] = pd.to_datetime(students_df['dob'])

# Calculate age in years
current_date = dt.datetime.now()
students_df['age'] = ((current_date - pd.to_datetime(students_df['dob'])).dt.days // 365.25).astype(int)

# Calculate age group by decade
students_df['age_group'] = (students_df['age'] // 10 * 10).astype(int)

students_df.head()

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,age_group
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99,81,80
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4,33,30
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74,34,30
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31,35,30
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64,60,60


### Rename `dob`

Renaming `dob` to `date_of_birth` will allow for better readability of the dataset, and make it easier to understand.

In [28]:
# Rename columns
students_df.rename(columns={'dob': 'date_of_birth'}, inplace=True)

students_df.head()

,uuid,name,date_of_birth,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,age_group
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99,81,80
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4,33,30
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74,34,30
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31,35,30
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64,60,60


### Split `contact_info` into separate fields

Splitting `contact_info` into separate fields would allow for more granualar analysis, better readability and make it easier to maintain data integrity.

In [29]:
students_df['contact_info'] = students_df["contact_info"].apply(lambda x: ast.literal_eval(x))
explode_contact = pd.json_normalize(students_df['contact_info'])
students_df = pd.concat([students_df.drop('contact_info', axis=1), explode_contact], axis=1)

students_df.head()

,uuid,name,date_of_birth,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,age_group,mailing_address,email
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,81,80,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,33,30,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,34,30,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,35,30,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,60,60,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com


### Split `mailing_address`

Splitting the mailing address into separate fields would allow for more detailed analysis into particular states and cities people might be unsubscribing from.

In [30]:
# Separate lines of mailing address
students_df['address_line_1'] = students_df['mailing_address'].str.split(',').str[0].str.strip()
students_df['city'] = students_df['mailing_address'].str.split(',').str[1].str.strip()
students_df["state"] = students_df['mailing_address'].str.split(',').str[2].str.strip()
students_df["zip_code"] = students_df['mailing_address'].str.split(',').str[3].str.strip()

students_df.drop(columns="mailing_address", inplace=True)

students_df.head()

,uuid,name,date_of_birth,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,age_group,email,address_line_1,city,state,zip_code
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,81,80,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,33,30,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,34,30,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,35,30,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,60,60,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737


### Handle Missing Values

The values for our dataset that are missing are:
- `job_id`
- `num_course_taken`
- `current_career_path_id`
- `time_spent_hrs`

For these records, I feel like they are missing values, as they did not undertake any courses after subscribing or did not have a job. As such, they would have no values for these fields, including the foreign keys `job_id` and `current_career_path_id`. These can all be filled with 0, but I will need to check the unique values for `job_id` and `current_career_path_id`, to make sure that this value has not been taken.

In [31]:
# Count unique job id's
students_df.job_id.value_counts()

job_id
2.0    706
1.0    693
7.0    680
3.0    675
4.0    671
5.0    660
6.0    657
8.0    253
Name: count, dtype: int64

In [32]:
# Count unique current career path id's
students_df.current_career_path_id.value_counts()

current_career_path_id
5.0     476
3.0     469
10.0    460
1.0     459
6.0     454
2.0     450
7.0     449
9.0     441
8.0     437
4.0     434
Name: count, dtype: int64

In [33]:
# Give null values a value of 0
students_df = students_df.fillna(0)

# Count unique current career path id's
students_df.current_career_path_id.value_counts()

current_career_path_id
5.0     476
0       471
3.0     469
10.0    460
1.0     459
6.0     454
2.0     450
7.0     449
9.0     441
8.0     437
4.0     434
Name: count, dtype: int64

### Change `job_id`, `num_course_taken`, `current_career_path_id` datatypes to integers

In [34]:
# Change datatypes from string to float
students_df['job_id'] = students_df['job_id'].astype(float)
students_df["num_course_taken"] = students_df["num_course_taken"].astype(float)
students_df["current_career_path_id"] = students_df["current_career_path_id"].astype(float)

# Change datatypes from float to int
students_df['job_id'] = students_df['job_id'].astype(int)
students_df["num_course_taken"] = students_df["num_course_taken"].astype(int)
students_df["current_career_path_id"] = students_df["current_career_path_id"].astype(int)

# verify
students_df.dtypes

uuid                               int64
name                              object
date_of_birth             datetime64[ns]
sex                               object
job_id                             int64
num_course_taken                   int64
current_career_path_id             int64
time_spent_hrs                    object
age                                int64
age_group                          int64
email                             object
address_line_1                    object
city                              object
state                             object
zip_code                          object
dtype: object

### Change `time_spent_hrs` dtype

Changing this to a time datatype will allow for better manipulability during analysis.

In [35]:
# Convert to float
students_df['time_spent_hrs'] = students_df['time_spent_hrs'].astype(float)

# Convert to timedelta
students_df['time_spent'] = pd.to_timedelta(students_df['time_spent_hrs'], unit='h')

# Drop time_spent_hrs
students_df.drop(columns="time_spent_hrs", inplace=True)

# verify
students_df.head()

,uuid,name,date_of_birth,sex,job_id,num_course_taken,current_career_path_id,age,age_group,email,address_line_1,city,state,zip_code,time_spent
0,1,Annabelle Avery,1943-07-03,F,7,6,1,81,80,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736,0 days 04:59:24
1,2,Micah Rubio,1991-02-07,M,7,5,8,33,30,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439,0 days 04:24:00
2,3,Hosea Dale,1989-12-07,M,7,8,8,34,30,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637,0 days 06:44:24
3,4,Mariann Kirk,1988-07-31,F,6,7,9,35,30,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242,0 days 12:18:36
4,5,Lucio Alexander,1963-08-31,M,7,14,3,60,60,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737,0 days 05:38:24


### Split `name`

Splitting the name will allow for greater granuality in analysis, and easier maintanance of data integrity.

In [36]:
students_df['first_name'] = students_df['name'].str.split(' ').str[0]
students_df['last_name'] = students_df['name'].str.split(' ').str[1]
students_df.drop(columns="name", inplace=True)
students_df.head()

,uuid,date_of_birth,sex,job_id,num_course_taken,current_career_path_id,age,age_group,email,address_line_1,city,state,zip_code,time_spent,first_name,last_name
0,1,1943-07-03,F,7,6,1,81,80,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736,0 days 04:59:24,Annabelle,Avery
1,2,1991-02-07,M,7,5,8,33,30,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439,0 days 04:24:00,Micah,Rubio
2,3,1989-12-07,M,7,8,8,34,30,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637,0 days 06:44:24,Hosea,Dale
3,4,1988-07-31,F,6,7,9,35,30,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242,0 days 12:18:36,Mariann,Kirk
4,5,1963-08-31,M,7,14,3,60,60,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737,0 days 05:38:24,Lucio,Alexander


## Courses Table

Now that I've transformed the students table, let's investigate the courses table.

In [37]:
# look at head
courses_df.head(10)

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


In [38]:
# look at dtypes
courses_df.dtypes

career_path_id        int64
career_path_name     object
hours_to_complete     int64
dtype: object

All datatypes appear to be correct. The main thing I would concern myself with here is adding an extra record to account for unsubscribers who did not sign up to a career path. Based on how I filled these in `students_df`, I would provide a `career_path_id` of `0`, and a name that makes it obvious that no path was taken.

In [39]:
# add row
na = {
    'career_path_id': 0,
    'career_path_name': 'not applicable',
    'hours_to_complete': 0
}
courses_df.loc[len(courses_df)] = na

# look at tail
courses_df.tail()

,career_path_id,career_path_name,hours_to_complete
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15
10,0,not applicable,0


## Jobs Table

Lastly, I will investigate `student_jobs_df`.

In [40]:
# look at head
student_jobs_df.head(10)  

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


It seems like those flagged with the `0` for the `job_id` would be given a job_category of `other`. This would be a good descriptor of those unknown.

## Joining Tables

In [41]:
final_df = students_df.merge(courses_df, left_on='current_career_path_id', right_on='career_path_id', how='left')
final_df = final_df.merge(student_jobs_df, on='job_id', how='left')
final_df.head()

,uuid,date_of_birth,sex,job_id,num_course_taken,current_career_path_id,age,age_group,email,address_line_1,...,state,zip_code,time_spent,first_name,last_name,career_path_id,career_path_name,hours_to_complete,job_category,avg_salary
0,1,1943-07-03,F,7,6,1,81,80,annabelle_avery9376@woohoo.com,303 N Timber Key,...,Wisconsin,84736,0 days 04:59:24,Annabelle,Avery,1,data scientist,20,HR,80000
1,2,1991-02-07,M,7,5,8,33,30,rubio6772@hmail.com,767 Crescent Fair,...,Indiana,37439,0 days 04:24:00,Micah,Rubio,8,android developer,27,HR,80000
2,3,1989-12-07,M,7,8,8,34,30,hosea_dale8084@coldmail.com,P.O. Box 41269,...,Virginia,83637,0 days 06:44:24,Hosea,Dale,8,android developer,27,HR,80000
3,4,1988-07-31,F,6,7,9,35,30,kirk4005@hmail.com,517 SE Wintergreen Isle,...,Arkansas,82242,0 days 12:18:36,Mariann,Kirk,9,machine learning engineer,35,education,61000
4,5,1963-08-31,M,7,14,3,60,60,alexander9810@hmail.com,18 Cinder Cliff,...,Rhode Island,73737,0 days 05:38:24,Lucio,Alexander,3,data analyst,12,HR,80000


In [42]:
final_df.info()
con.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7006 entries, 0 to 7005
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   uuid                    7006 non-null   int64          
 1   date_of_birth           7006 non-null   datetime64[ns] 
 2   sex                     7006 non-null   object         
 3   job_id                  7006 non-null   int64          
 4   num_course_taken        7006 non-null   int64          
 5   current_career_path_id  7006 non-null   int64          
 6   age                     7006 non-null   int64          
 7   age_group               7006 non-null   int64          
 8   email                   7006 non-null   object         
 9   address_line_1          7006 non-null   object         
 10  city                    7006 non-null   object         
 11  state                   7006 non-null   object         
 12  zip_code                7006 non-n

## Add to New SQL Table

In [43]:
con = sqlite3.connect('data/cademycode_cleansed.db')
final_df.to_sql('cademycode_aggregated', con, if_exists='replace', index=False)
final_df.to_csv('data/cademycode_aggregated.csv', index=False)

con.commit()
con.close()

C:\Users\charl.DESKTOP-1NGH5IT\AppData\Local\Temp\ipykernel_13144\803380638.py:2: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  final_df.to_sql('cademycode_aggregated', con, if_exists='replace', index=False)
